# Taxpayer Risk Classification Project

## Business Problem
Tax authorities face significant challenges in efficiently identifying taxpayers who are at high risk of non-compliance or tax evasion. Without accurate risk profiling, audits and enforcement actions may be misallocated, leading to revenue losses and wasted resources. There is a need for a data-driven approach to classify taxpayers based on their risk levels, enabling focused compliance efforts.

## Stakeholders
| Stakeholder             | Role / Interest                                                                                 |
|------------------------|------------------------------------------------------------------------------------------------|
| Tax Authority / Revenue Service | Responsible for tax collection, compliance enforcement, and overall revenue maximization.    |
| Audit Teams             | Use risk classifications to prioritize audits and investigations for efficient resource use.   |
| Policy Makers           | Use insights from the model to improve tax regulations and compliance strategies.               |
| Taxpayers               | Subject to audits and compliance monitoring; directly impacted by classification outcomes.     |
| Data Analysts / Data Scientists | Develop, validate, and maintain predictive models, providing actionable insights to stakeholders. |


## Business Objectives
1. **How can we improve audit efficiency by prioritizing audits on high-risk taxpayers to reduce costs and increase revenue recovery?**

2. **How can we develop a predictive model to categorize taxpayers by risk level (Low, Medium, High) for ongoing compliance monitoring and early intervention?**

3. **How can we proactively identify potential non-compliant taxpayers to reduce tax evasion and maximize government revenue?**

## Analysis Objectives
- **How can we build and validate a machine learning classification model that accurately predicts the risk label of taxpayers based on their financial and behavioral features?**

- **Which features (e.g., revenue, expenses, late filings) most significantly influence the taxpayer risk classification and how can these insights inform targeted policy actions?**

## Data  Description
| Column Name          | Description                                                                                       | Data Type              |
|----------------------|---------------------------------------------------------------------------------------------------|------------------------|
| Taxpayer_ID          | Unique identifier for each taxpayer                                                               | Categorical (ID)       |
| Revenue              | Total revenue reported by the taxpayer over a specific period                                     | Numeric (Continuous)   |
| Expenses             | Total expenses declared by the taxpayer                                                           | Numeric (Continuous)   |
| Profit               | Net profit calculated as Revenue minus Expenses                                                   | Numeric (Continuous)   |
| Industry_Type        | Sector in which the taxpayer operates (e.g., Retail, Manufacturing)                              | Categorical            |
| Business_Age         | Number of years the business has been operating                                                   | Numeric (Discrete)     |
| Late_Filings         | Number of late tax return submissions                                                              | Numeric (Discrete)     |
| Audit_History        | Count of previous audits conducted on the taxpayer                                                | Numeric (Discrete)     |
| Payment_Delays       | Number of instances where tax payments were delayed                                               | Numeric (Discrete)     |
| Employee_Count       | Number of employees reported by the business                                                      | Numeric (Discrete)     |
| Compliance_Score     | Internal score indicating taxpayer’s level of compliance (e.g., 0–100 scale)                      | Numeric (Continuous)   |
| Region               | Geographical location or jurisdiction of the taxpayer                                             | Categorical            |
| Risk_Label           | Target variable: classification of the taxpayer’s compliance risk (Low, Medium, High)            | Categorical (Target)   |



## Prediction Target
We are predicting the **Taxpayer Risk Label**, which categorizes taxpayers into different risk levels **Low**, **Medium**, or **High** risk of non-compliance or tax evasion based on their financial data and compliance behavior. This classification helps tax authorities prioritize audits and enforcement actions effectively.


# Import modules & packages

In [2]:
# Data manipulation 
import pandas as pd 
import numpy as np 

# Data visualization
import seaborn as sns 
import matplotlib.pyplot as plt 

# Modeling
from sklearn.model_selection import train_test_split,GridSearchCV
from imblearn.over_sampling import SMOTE #SMOTE technique to deal with unbalanced data problem
from sklearn.metrics import accuracy_score,f1_score,recall_score,precision_score,confusion_matrix,roc_curve,auc,classification_report # performance metrics
from sklearn.preprocessing import StandardScaler# to scale the numeric features
from sklearn.preprocessing import LabelEncoder,OneHotEncoder,StandardScaler # to encode binary features
from scipy import stats

# Algorithms for supervised learning methods
from sklearn.tree import DecisionTreeClassifier,plot_tree
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import ConfusionMatrixDisplay


# Set the style to "darkgrid" and "ggplot"
sns.set_style("dark")

In [3]:

data = pd.read_csv(r"C:\Users\Harriet\Downloads\New folder (2)\tax_risk_dataset.csv")
data.head()

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Harriet\\Downloads\\New folder (2)\\tax_risk_dataset.csv'

# EXPLORATORY DATA ANALYSIS

# DATA INSPECTION

In [ ]:
print("\n Column names:")
print(data.columns.tolist())

print("\n Data types and non-null counts:")
print(data.info())

print("\n Summary statistics (numerical columns):")
print(data.describe())

print("\n Preview of the first 5 rows:")
print(data.head())

print("\n Preview rows and columns:")
print(data.shape)


 Column names:
['Taxpayer_ID', 'Revenue', 'Expenses', 'Tax_Liability', 'Tax_Paid', 'Late_Filings', 'Compliance_Violations', 'Industry', 'Profit', 'Tax_Compliance_Ratio', 'Audit_Findings', 'Audit_to_Tax_Ratio', 'Risk_Label']

 Data types and non-null counts:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Taxpayer_ID            1000 non-null   int64  
 1   Revenue                1000 non-null   float64
 2   Expenses               1000 non-null   float64
 3   Tax_Liability          1000 non-null   float64
 4   Tax_Paid               1000 non-null   float64
 5   Late_Filings           1000 non-null   int64  
 6   Compliance_Violations  1000 non-null   int64  
 7   Industry               1000 non-null   object 
 8   Profit                 1000 non-null   float64
 9   Tax_Compliance_Ratio   1000 non-null   float64
 10  Audit_

In [ ]:
# hello